# le bon script

In [1]:
import requests
import pandas as pd
import sqlite3
import os
from io import StringIO
from bs4 import BeautifulSoup

# L'URL du site web
url = "https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/"

# Dossier de destination pour les fichiers CSV
download_folder = "csv_files"
os.makedirs(download_folder, exist_ok=True)

# Créer une base de données SQLite
db_conn = sqlite3.connect("air_quality_data.db")

# Envoyer une requête GET pour obtenir le contenu de la page
response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Analyser le contenu de la page avec BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Trouver les liens des fichiers CSV
    links = soup.find_all('a', href=True)
    csv_links = [link['href'] for link in links if link['href'].endswith('.csv')]

    for csv_link in csv_links:
        csv_url = url + csv_link
        response_csv = requests.get(csv_url)

        if response_csv.status_code == 200:
            # Utiliser StringIO correctement en lisant le contenu binaire
            content = response_csv.content.decode('utf-8-sig')
            df = pd.read_csv(StringIO(content), sep=';')

            # Renommer la colonne "Date de début" en "Date"
            df.rename(columns={"Date de début": "Date"}, inplace=True)

            # Supprimer la colonne "Date de fin" si elle existe
            if "Date de fin" in df.columns:
                df.drop(columns=["Date de fin"], inplace=True)

            # Renommer la colonne "Zas" en "ZAS" pour correspondre à la table SQLite
            df.rename(columns={"Zas": "ZAS"}, inplace=True)

            # Créer une table dans la base de données avec le schéma du DataFrame
            df.to_sql('air_quality', db_conn, if_exists='replace', index=False)

# Fermer la connexion à la base de données
db_conn.close()


### Affichage de la table 

In [39]:
import sqlite3

# Connexion à la base de données SQLite
db_conn = sqlite3.connect("air_quality_data.db")

# Création d'un curseur pour exécuter des requêtes SQL
cursor = db_conn.cursor()

# Exécution d'une requête SQL pour sélectionner toutes les lignes de la table air_quality
cursor.execute("SELECT * FROM air_quality")

# Récupération de toutes les lignes de résultat
rows = cursor.fetchall()

# Affichage des données
for row in rows:
    print(row)

# Fermeture du curseur et de la connexion à la base de données
cursor.close()
db_conn.close()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [43]:
import sqlite3

# Connexion à la base de données SQLite
db_conn = sqlite3.connect("air_quality_data.db")

# Création d'un curseur pour exécuter des requêtes SQL
cursor = db_conn.cursor()

# Exécution d'une requête SQL pour sélectionner toutes les lignes de la table air_quality
cursor.execute("SELECT * FROM air_quality")

# Récupération de toutes les lignes de résultat
rows = cursor.fetchall()

# Fermeture du curseur et de la connexion à la base de données
cursor.close()
db_conn.close()

# Nombre de lignes par page
rows_per_page = 10

# Affichage des données par pages
page_number = 1

while True:
    start_index = (page_number - 1) * rows_per_page
    end_index = start_index + rows_per_page

    page_data = rows[start_index:end_index]

    if not page_data:
        break

    # Affichage des données de la page
    for row in page_data:
        print(row)

    # Demande à l'utilisateur s'il souhaite afficher la page suivante
    user_input = input("Afficher la page suivante (O/N) ? ").strip().lower()

    if user_input != 'o':
        break

    page_number += 1


('2021/12/31 00:00:00', 'ATMO GRAND EST', 'FR44ZAG02', 'ZAG METZ', 'FR01011', 'Metz-Centre', 'Urbaine', 'NO', 'Fond', 'A', 'Oui', 'mesures fixes', 'Auto NO Conf meth CHIMILU', 'moyenne horaire validée', 0.7, 0.675, 'µg-m3', None, None, None, 'A', 1)
('2021/12/31 01:00:00', 'ATMO GRAND EST', 'FR44ZAG02', 'ZAG METZ', 'FR01011', 'Metz-Centre', 'Urbaine', 'NO', 'Fond', 'A', 'Oui', 'mesures fixes', 'Auto NO Conf meth CHIMILU', 'moyenne horaire validée', 0.7, 0.725, 'µg-m3', None, None, None, 'A', 1)
('2021/12/31 02:00:00', 'ATMO GRAND EST', 'FR44ZAG02', 'ZAG METZ', 'FR01011', 'Metz-Centre', 'Urbaine', 'NO', 'Fond', 'A', 'Oui', 'mesures fixes', 'Auto NO Conf meth CHIMILU', 'moyenne horaire validée', 0.9, 0.85, 'µg-m3', None, None, None, 'A', 1)
('2021/12/31 03:00:00', 'ATMO GRAND EST', 'FR44ZAG02', 'ZAG METZ', 'FR01011', 'Metz-Centre', 'Urbaine', 'NO', 'Fond', 'A', 'Oui', 'mesures fixes', 'Auto NO Conf meth CHIMILU', 'moyenne horaire validée', 0.8, 0.825, 'µg-m3', None, None, None, 'A', 1)
(

KeyboardInterrupt: Interrupted by user

In [ ]:
import sqlite3
import csv

# Connexion à la base de données SQLite
db_conn = sqlite3.connect("air_quality_data.db")

# Création d'un curseur pour exécuter des requêtes SQL
cursor = db_conn.cursor()

# Exécution d'une requête SQL pour sélectionner toutes les lignes de la table air_quality
cursor.execute("SELECT * FROM air_quality")

# Récupération de toutes les lignes de résultat
rows = cursor.fetchall()

# Fermeture du curseur et de la connexion à la base de données
cursor.close()
db_conn.close()

# Nom du fichier CSV de sortie
output_file = "air_quality_export.csv"

# Écriture des données dans le fichier CSV
with open(output_file, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Écriture de l'en-tête (noms de colonnes)
    column_names = [description[0] for description in cursor.description]
    csv_writer.writerow(column_names)

    # Écriture des données
    for row in rows:
        csv_writer.writerow(row)

print(f"Les données ont été exportées avec succès vers {output_file}")


In [41]:
# Afficher les premières lignes du DataFrame airparif_data
print(airparif_data.head(100000))

                     Date Organisme   code zas        ZAS code site  \
3672  2021/01/01 00:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04002   
3673  2021/01/01 01:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04002   
3674  2021/01/01 02:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04002   
3675  2021/01/01 03:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04002   
3676  2021/01/01 04:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04002   
...                   ...       ...        ...        ...       ...   
7987  2021/01/01 19:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04329   
7988  2021/01/01 20:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04329   
7989  2021/01/01 21:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04329   
7990  2021/01/01 22:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04329   
7991  2021/01/01 23:00:00  AIRPARIF  FR11ZAG01  ZAG PARIS   FR04329   

                  nom site type d'implantation Polluant type d'influence  \
3672         GENNEVILLIERS             Urbaine       NO             Fon

In [42]:
pd.set_option('display.max_rows', 100000)  # Définir le nombre maximal de lignes à afficher
print(airparif_data)

                     Date Organisme   code zas               ZAS code site  \
3672  2021/01/01 00:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3673  2021/01/01 01:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3674  2021/01/01 02:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3675  2021/01/01 03:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3676  2021/01/01 04:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3677  2021/01/01 05:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3678  2021/01/01 06:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3679  2021/01/01 07:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3680  2021/01/01 08:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3681  2021/01/01 09:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3682  2021/01/01 10:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS   FR04002   
3683  2021/01/01 11:00:00  AIRPARIF  FR11ZAG01         ZAG PARIS

In [3]:
### traitement de données avec Pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext

# Créer une session Spark
spark = SparkSession.builder.appName("AirQualityAnalysis").getOrCreate()

# Charger les données depuis la base de données SQLite
air_quality_data = spark.read.format("db").option("url", "db:sqlite:air_quality_data.db").option("dbtable", "air_quality").load()

# Afficher le schéma des données
air_quality_data.printSchema()

# Afficher les premières lignes de données
air_quality_data.show()

# Effectuer une analyse simple, par exemple, calculer la moyenne des concentrations de polluants
average_concentration = air_quality_data.groupBy("Pollutant").agg(mean("Concentration").alias("Average_Concentration"))

# Afficher les résultats
average_concentration.show()

# Vous pouvez effectuer d'autres opérations d'analyse ici, comme la corrélation, la régression, etc.

# Enfin, n'oubliez pas de fermer la session Spark
spark.stop()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.